In [ ]:
import yfinance as yf

bpac3 = yf.Ticker("BPAC3.SA")
df_bpac3 = bpac3.history(start="2021-01-01",end="2023-12-30")

In [ ]:

from sklearn.metrics import root_mean_squared_error
import pandas as pd
import plotly.express as px
from statsmodels.tsa.arima.model import ARIMA

train_size_bpac3 = int(len(df_bpac3) * 0.8)
train_bpac3, test_bpac3 = df_bpac3[0:train_size_bpac3], df_bpac3[train_size_bpac3:len(df_bpac3)]

matrix = {"janela":[], "rmse_a": [], "rmse_e":[], "rmse_arima":[]}
for i in range(2, 15):
    # p - Número de observações passadas
    # d - Número de vezes que ocorreram diferenciação entre os dados passados 
    # q - Janela de médias móveis
    model_train = ARIMA(train_bpac3['Close'], order=(1, 1, i))
    
    model_train_fit = model_train.fit()

    test_forecast = model_train_fit.get_forecast(steps=len(test_bpac3))
    test_forecast_series = pd.Series(test_forecast.predicted_mean.to_numpy(), index=test_bpac3.index)

    # Calculate the mean squared error
    rmse_arima = root_mean_squared_error(test_bpac3['Close'], test_forecast_series)
    name_a = f'med_mov_aritimética_{i}'
    name_e = f'med_mov_exponencial_{i}'
    df_bpac3[name_a] = df_bpac3['Close'].rolling(window=i, min_periods=0).mean()
    df_bpac3[name_e] = df_bpac3['Close'].ewm(span=i, min_periods=0,adjust=False).mean()
    rmse_a = root_mean_squared_error(df_bpac3['Close'].to_numpy(), df_bpac3[name_a].to_numpy())
    rmse_e = root_mean_squared_error(df_bpac3['Close'].to_numpy(), df_bpac3[name_e].to_numpy())
    matrix["janela"].append(i)
    matrix["rmse_a"].append(rmse_a.round(5))
    matrix["rmse_e"].append(rmse_e.round(5))
    matrix['rmse_arima'].append(rmse_arima.round(5))



In [ ]:


df = pd.DataFrame(matrix)

fig = px.line(
    df, 
    x='janela',
    y=['rmse_a', 'rmse_e', 'rmse_arima'],
    
)
fig.update_layout(title="Relação RMSE x Qde de dias analisados para BPAC3", xaxis_title="Qde de dias analisados", yaxis_title="RMSE") 

fig.show()